# Exploratory Data Analysis Notebook
    -   This notebook to download raw market data, clean data and add features for machine leanring modeling.

## Steps: 
    - Download raw market data 
    - Add data to the database: Arcticdb
    - Clean data of NaN, duplicate values
    - Create tables of the data
    - Save the cleaned data into the processed folder

In [ ]:
import os
import pathlib
import sys
sys.path.append(str(pathlib.Path(os.getcwd()).parent))

In [ ]:
from src.data_ingest import DataIngestor
from src.feature_engineering import FeatureEngineer

In [ ]:
Equity_portfolio = ['AAPL',   # Apple
    'MSFT', # Microsoft
    'AMZN', # Amazon
    'GOOGL',# Alphabet 
    'TSLA', # Tesla
    'NFLX', # Netflix
    'NVDA', # NVIDIA
    'META'] # Meta Platforms

ETF_portfolio = ['SPY', # S&P 500
    'QQQ', # NASDAQ 100
    'IWM', # Russell 2000
    'DIA', # Dow Jones Industrial Average
    'VTI', # Vanguard Total Stock Market
    'EEM', # Emerging Markets
    'ARKK', # ARK Innovation
    'GLD', # SPDR Gold Shares
    'SLV'] # iShares Silver Trust

Sector_ETF = ['XLF', # Financials
    'XLK', # Technology
    'XLY', # Consumer Discretionary
    'XLE', # Energy
    'XLV', # Health Care
    'XLI', # Industrials
    'XLB', # Materials
    'XLU'] # Utilities

In [ ]:
data_ingestor = DataIngestor(db_uri="lmdb://data/arcticdb")
feature_engineer = FeatureEngineer(long = 50, short = 20)

In [ ]:
# Ingest Equity Portfolio data
print("=" * 60)
print("Ingesting Equity Portfolio")
print("=" * 60)
for ticker in Equity_portfolio:
    data_ingestor.pipeline_summary(ticker, start_date="2015-01-01", end_date="2024-12-31", collection_name=f"Equity_{ticker}")
    print()

# Ingest ETF Portfolio data
print("=" * 60)
print("Ingesting ETF Portfolio")
print("=" * 60)
for ticker in ETF_portfolio:
    data_ingestor.pipeline_summary(ticker, start_date="2015-01-01", end_date="2024-12-31", collection_name=f"ETF_{ticker}")
    print()

# Ingest Sector ETF data
print("=" * 60)
print("Ingesting Sector ETF Portfolio")
print("=" * 60)
for ticker in Sector_ETF:
    data_ingestor.pipeline_summary(ticker, start_date="2015-01-01", end_date="2024-12-31", collection_name=f"Sector_{ticker}")
    print()

In [ ]:
# List all collections in the database
data_ingestor.list_collections()

## Display Sample Data

Let's look at some sample data from one of the equity stocks:

In [ ]:
# Display data for AAPL
data_ingestor.display_data(collection_name="Equity_AAPL", head=10)

## Create and Save Tables

Now let's create tables for all the portfolios and save them to Excel files:

In [ ]:
# Create and save tables for Equity Portfolio
print("Creating tables for Equity Portfolio...")
for ticker in Equity_portfolio:
    data_ingestor.create_and_save_all_tables(
        collection_name=f"Equity_{ticker}",
        output_dir="results/tables/equity"
    )
    print()

In [ ]:
# Create and save tables for ETF Portfolio
print("Creating tables for ETF Portfolio...")
for ticker in ETF_portfolio:
    data_ingestor.create_and_save_all_tables(
        collection_name=f"ETF_{ticker}",
        output_dir="results/tables/etf"
    )
    print()

In [ ]:
# Create and save tables for Sector ETF Portfolio
print("Creating tables for Sector ETF Portfolio...")
for ticker in Sector_ETF:
    data_ingestor.create_and_save_all_tables(
        collection_name=f"Sector_{ticker}",
        output_dir="results/tables/sector"
    )
    print()

## Read and Analyze Data

Now let's read some data back from the database for analysis:

In [ ]:
# Read AAPL data from the database
aapl_data = data_ingestor.read_data(collection_name="Equity_AAPL")
print(f"AAPL Data Shape: {aapl_data.shape}")
print(f"\nFirst few rows:")
aapl_data.head()

In [ ]:
# Summary statistics
aapl_data.describe()

## Save Cleaned data to Processed folder 

In [ ]:
# Save cleaned data to processed folder
import os

# Create processed data directory if it doesn't exist
processed_dir = "data/processed"
os.makedirs(processed_dir, exist_ok=True)

# Save Equity Portfolio data
print("Saving Equity Portfolio to processed folder...")
for ticker in Equity_portfolio:
    df = data_ingestor.read_data(collection_name=f"Equity_{ticker}")
    output_path = os.path.join(processed_dir, f"Equity_{ticker}_cleaned.csv")
    df.to_csv(output_path)
    print(f"  ✓ Saved {ticker} to {output_path}")

# Save ETF Portfolio data
print("\nSaving ETF Portfolio to processed folder...")
for ticker in ETF_portfolio:
    df = data_ingestor.read_data(collection_name=f"ETF_{ticker}")
    output_path = os.path.join(processed_dir, f"ETF_{ticker}_cleaned.csv")
    df.to_csv(output_path)
    print(f"  ✓ Saved {ticker} to {output_path}")

# Save Sector ETF data
print("\nSaving Sector ETF Portfolio to processed folder...")
for ticker in Sector_ETF:
    df = data_ingestor.read_data(collection_name=f"Sector_{ticker}")
    output_path = os.path.join(processed_dir, f"Sector_{ticker}_cleaned.csv")
    df.to_csv(output_path)
    print(f"  ✓ Saved {ticker} to {output_path}")

print(f"\n✓ All cleaned data saved to '{processed_dir}' folder!")

## Data Visualization

Let's visualize some of the data to understand patterns and trends:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 7)

In [ ]:
# Plot closing prices for all equity stocks
plt.figure(figsize=(16, 8))

for ticker in Equity_portfolio:
    df = data_ingestor.read_data(collection_name=f"Equity_{ticker}")
    close_col = [col for col in df.columns if 'Close' in col][0]
    plt.plot(df.index, df[close_col], label=ticker, linewidth=2)

plt.title('Equity Portfolio - Closing Prices Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Price ($)', fontsize=12)
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Calculate and plot daily returns for AAPL
aapl_close = [col for col in aapl_data.columns if 'Close' in col][0]
aapl_returns = aapl_data[aapl_close].pct_change()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Plot returns
axes[0].plot(aapl_returns.index, aapl_returns, linewidth=0.8, color='blue', alpha=0.7)
axes[0].set_title('AAPL Daily Returns', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date', fontsize=12)
axes[0].set_ylabel('Returns', fontsize=12)
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0].grid(True, alpha=0.3)

# Plot returns distribution
axes[1].hist(aapl_returns.dropna(), bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[1].set_title('AAPL Returns Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Returns', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].axvline(x=0, color='r', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"AAPL Returns Statistics:")
print(f"Mean: {aapl_returns.mean():.6f}")
print(f"Std Dev: {aapl_returns.std():.6f}")
print(f"Min: {aapl_returns.min():.6f}")
print(f"Max: {aapl_returns.max():.6f}")

In [ ]:
# Volume analysis
aapl_volume = [col for col in aapl_data.columns if 'Volume' in col][0]

plt.figure(figsize=(16, 6))
plt.bar(aapl_data.index, aapl_data[aapl_volume], color='teal', alpha=0.6, width=1)
plt.title('AAPL Trading Volume Over Time', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Volume', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Correlation Analysis

Let's analyze the correlations between different stocks:

In [ ]:
# Create a DataFrame with closing prices for all equity stocks
import pandas as pd

equity_closes = pd.DataFrame()

for ticker in Equity_portfolio:
    df = data_ingestor.read_data(collection_name=f"Equity_{ticker}")
    close_col = [col for col in df.columns if 'Close' in col][0]
    equity_closes[ticker] = df[close_col]

# Calculate correlation matrix
correlation_matrix = equity_closes.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Equity Portfolio Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nHighest Correlations:")
# Get upper triangle of correlation matrix
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
correlation_matrix_masked = correlation_matrix.where(~mask)
# Find top correlations
correlations = correlation_matrix_masked.unstack().sort_values(ascending=False)
print(correlations.head(10))

In [ ]:
# Calculate returns correlation
equity_returns = equity_closes.pct_change().dropna()
returns_correlation = equity_returns.corr()

plt.figure(figsize=(12, 10))
sns.heatmap(returns_correlation, annot=True, fmt='.2f', cmap='RdYlGn', 
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, center=0)
plt.title('Equity Portfolio Returns Correlation Matrix', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

## Risk-Return Analysis

Calculate and visualize the risk-return profile of each stock:

In [ ]:
# Calculate risk (std) and return (mean) for each stock
risk_return = pd.DataFrame({
    'Expected Return': equity_returns.mean() * 252,  # Annualized
    'Volatility (Risk)': equity_returns.std() * np.sqrt(252)  # Annualized
})

# Plot risk-return scatter
plt.figure(figsize=(14, 8))
for ticker in risk_return.index:
    plt.scatter(risk_return.loc[ticker, 'Volatility (Risk)'], 
                risk_return.loc[ticker, 'Expected Return'],
                s=200, alpha=0.6)
    plt.annotate(ticker, 
                 (risk_return.loc[ticker, 'Volatility (Risk)'], 
                  risk_return.loc[ticker, 'Expected Return']),
                 fontsize=12, fontweight='bold')

plt.title('Risk-Return Profile (Annualized)', fontsize=16, fontweight='bold')
plt.xlabel('Volatility (Risk)', fontsize=12)
plt.ylabel('Expected Return', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nRisk-Return Statistics:")
print(risk_return.sort_values('Expected Return', ascending=False))

## Feature Engineering

Apply technical indicators and feature engineering:

In [ ]:
# Apply feature engineering to AAPL data
# First, we need to check what methods are available in FeatureEngineer
print("Applying technical indicators to AAPL data...")

# Get AAPL data with simplified column names for feature engineering
aapl_fe = aapl_data.copy()

# Rename columns to match what feature_engineer expects (without ticker suffix)
column_mapping = {}
for col in aapl_fe.columns:
    if 'Close' in col:
        column_mapping[col] = 'Close'
    elif 'Open' in col:
        column_mapping[col] = 'Open'
    elif 'High' in col:
        column_mapping[col] = 'High'
    elif 'Low' in col:
        column_mapping[col] = 'Low'
    elif 'Volume' in col:
        column_mapping[col] = 'Volume'

aapl_fe = aapl_fe.rename(columns=column_mapping)

print(f"Original data shape: {aapl_fe.shape}")
print(f"Columns: {aapl_fe.columns.tolist()}")

## Summary and Conclusions

### Key Findings from EDA:

1. **Data Collection**: Successfully ingested and cleaned data for:
   - 8 Equity stocks (Tech giants)
   - 9 ETFs (Market indices and commodities)
   - 8 Sector ETFs

2. **Data Quality**: 
   - All data stored in ArcticDB database
   - No missing values after cleaning
   - Data saved to processed folder in CSV format
   - Tables exported to Excel for reporting

3. **Visualizations Created**:
   - Price trends over time
   - Returns analysis and distribution
   - Volume patterns
   - Correlation matrices
   - Risk-return profiles

4. **Next Steps**:
   - Apply regime detection models
   - Build risk forecasting models
   - Develop trading strategies
   - Backtest strategies

In [ ]:
# Final summary statistics
print("=" * 80)
print("EXPLORATORY DATA ANALYSIS - FINAL SUMMARY")
print("=" * 80)

print(f"\n📊 Total Collections in Database: {len(data_ingestor.list_collections())}")

print(f"\n📈 Equity Portfolio ({len(Equity_portfolio)} stocks):")
for ticker in Equity_portfolio:
    print(f"   • {ticker}")

print(f"\n📊 ETF Portfolio ({len(ETF_portfolio)} ETFs):")
for ticker in ETF_portfolio:
    print(f"   • {ticker}")

print(f"\n🏭 Sector ETF Portfolio ({len(Sector_ETF)} sectors):")
for ticker in Sector_ETF:
    print(f"   • {ticker}")

print("\n💾 Data Storage:")
print("   • Database: ArcticDB (LMDB)")
print("   • Processed files: data/processed/")
print("   • Tables: results/tables/")

print("\n✅ EDA Complete! Ready for modeling and strategy development.")
print("=" * 80)